In [ ]:
import pandas as pd
import datetime as dt
from google.colab import drive

In [ ]:
def getfile(location_pair,**kwargs): #tries to get local version and then defaults to google drive version
    (loc,gdrive)=location_pair
    try:
        out=pd.read_csv(loc,**kwargs)
    except FileNotFoundError:
        print("local file not found; accessing Google Drive")
        loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
        out=pd.read_csv(loc,**kwargs)
    return out

In [ ]:
local = False
fpath = 'drive/MyDrive/DeepDiveProject/data/'
if local:
  drive.mount('/content/drive')

actual_energy_price=(fpath + "20201019-20241020 MISO Actual Energy Price.csv","https://drive.google.com/file/d/1mlb-QV1MVL16LmxeGRFYnc6LZgRhaZER/view?usp=drive_link")
cleared_load=(fpath + "20231020-20241020 MISO Cleared Load.csv","https://drive.google.com/file/d/15uvhWbV9Tk12yEE22Kl0jovS2ciseiO5/view?usp=drive_link")
forecasted_load=(fpath + "20231019-20241020 MISO Forecasted Load.csv","https://drive.google.com/file/d/138MOchfLDEsj8U8PwUrkugiVJgBeyuxZ/view?usp=drive_link")
actual_load=(fpath + "20231019-20241020 MISO Actual Load.csv","https://drive.google.com/file/d/12hSzcFnpATD8upDGMVegMDgmlaZlH3zv/view?usp=drive_link")
avg_retail_price=(fpath + "Average_retail_price_of_electricity.csv","https://drive.google.com/file/d/1Wv17rfDyl33N1HZaEmIkfNRxDnweWlKe/view?usp=drive_link")
chicago_weather=(fpath + "20231019-20241020 Chicago Weather Data.csv","https://drive.google.com/file/d/18KleYA0hFGO9TB53cHF8KiDqOn7g7SO1/view?usp=drive_link")
champaign_weather=(fpath + "20231019-20241020 Champaign Weather Data.csv","https://drive.google.com/file/d/1CdH4_uSRksdOdeORgnsNn9lMq9x21LL5/view?usp=drive_link")
springfield_weather=(fpath + "20231019-20241020 Springfield Weather Data.csv","https://drive.google.com/file/d/1NbE6BPl_zvOZ49AxgsZRBcqDJWfpiLjT/view?usp=drive_link")

In [ ]:
data_files = {
    'actual_energy_price': actual_energy_price,
    'cleared_load': cleared_load,
    'forecasted_load': forecasted_load,
    'actual_load': actual_load,
    'retail_price': avg_retail_price,
    'chicago_weather': chicago_weather,
    'champaign_weather': champaign_weather,
    'springfield_weather': springfield_weather}

In [ ]:
# Get data
dfs = {
    name: getfile(f) for name, f in data_files.items()
}

local file not found; accessing Google Drive
local file not found; accessing Google Drive
local file not found; accessing Google Drive
local file not found; accessing Google Drive
local file not found; accessing Google Drive
local file not found; accessing Google Drive
local file not found; accessing Google Drive
local file not found; accessing Google Drive


In [ ]:
# Process each dataset
# Actual Energy Price - filter for Illinois HUB and rename column
dfs["actual_energy_price"] = dfs["actual_energy_price"][dfs["actual_energy_price"]['HUB'] == 'ILLINOIS.HUB'][['Date', 'LMP']]
dfs["actual_energy_price"].rename(columns={'LMP': 'Actual Price'}, inplace=True)

dfs["cleared_load"].rename(columns={'Load': 'Cleared Load'}, inplace=True)

dfs["forecasted_load"].rename(columns={'Load': 'Forecasted Load'}, inplace=True)

dfs["actual_load"].rename(columns={'Load': 'Actual Load'}, inplace=True)

# Retail Price filter and process index for date parsing
dfs["retail_price"] = dfs["retail_price"][dfs["retail_price"]['description'] == 'Residential : Illinois'].T.iloc[3:]
dfs["retail_price"].columns = ['Retail Price(cents/kwhr)']
dfs["retail_price"]['merge'] = pd.to_datetime(dfs["retail_price"].index, format='%b %Y').strftime('%m %y')

In [ ]:
# Create weather df
# Change column naming for merge for weather data for each city
cities = ['chicago', 'champaign', 'springfield']
for city in cities:
  for col in ['Max Temp (F)', 'Min Temp (F)', 'Mean Temp (F)']:
    dfs[city + '_weather'][col] = dfs[city + '_weather'][col].replace('M', 'nan')
    dfs[city + '_weather'][col] = dfs[city + '_weather'][col].astype('float')
  for col in ['Precipitation (in)', 'Snowfall (in)']:
    dfs[city + '_weather'][col] = dfs[city + '_weather'][col].replace('M', 'nan')
    dfs[city + '_weather'][col] = dfs[city + '_weather'][col].replace('T', '0.05')
    dfs[city + '_weather'][col] = dfs[city + '_weather'][col].astype('float')
  rename_dict = {
      'Precipitation (in)' : 'Precipitation (in) ' + city.capitalize(),
      'Snowfall (in)' : 'Snowfall (in) ' + city.capitalize(),
      'Max Temp (F)' : 'Max Temp (F) ' + city.capitalize(),
      'Min Temp (F)' : 'Min Temp (F) ' + city.capitalize(),
      'Mean Temp (F)' : 'Mean Temp (F) ' + city.capitalize(),
  }
  dfs[city + '_weather'].rename(columns=rename_dict, inplace=True)
  dfs[city + '_weather']['Date'] = pd.to_datetime(dfs[city + '_weather']['Date'])
  dfs[city + '_weather'].drop(columns=['Station'], inplace=True)

cities = ['Chicago', 'Champaign', 'Springfield']
weather_df = pd.merge(dfs['chicago_weather'], dfs['champaign_weather'], on='Date')
weather_df = pd.merge(weather_df, dfs['springfield_weather'], on='Date')
weather_df['Average Max Temp (F)'] = weather_df[['Max Temp (F) ' + c for c in cities]].mean(axis=1)
weather_df['Average Min Temp (F)'] = weather_df[['Min Temp (F) ' + c for c in cities]].mean(axis=1)
weather_df['Average Mean Temp (F)'] = weather_df[['Mean Temp (F) ' + c for c in cities]].mean(axis=1)
weather_df['Average Precipitation (in)'] = weather_df[['Precipitation (in) ' + c for c in cities]].mean(axis=1)
weather_df['Average Snowfall (in)'] = weather_df[['Snowfall (in) ' + c for c in cities]].mean(axis=1)
weather_df

,Date,Precipitation (in) Chicago,Snowfall (in) Chicago,Max Temp (F) Chicago,Min Temp (F) Chicago,Mean Temp (F) Chicago,Precipitation (in) Champaign,Snowfall (in) Champaign,Max Temp (F) Champaign,Min Temp (F) Champaign,...,Precipitation (in) Springfield,Snowfall (in) Springfield,Max Temp (F) Springfield,Min Temp (F) Springfield,Mean Temp (F) Springfield,Average Max Temp (F),Average Min Temp (F),Average Mean Temp (F),Average Precipitation (in),Average Snowfall (in)
0,2023-10-19,0.10,0.0,61.0,54.0,57.5,0.12,0.0,70.0,45.0,...,0.11,0.0,62.0,50.0,56.0,64.333333,49.666667,57.000000,0.110000,0.0
1,2023-10-20,0.00,0.0,61.0,48.0,54.5,0.25,0.0,59.0,46.0,...,0.00,0.0,68.0,45.0,56.5,62.666667,46.333333,54.500000,0.083333,0.0
2,2023-10-21,0.02,0.0,65.0,47.0,56.0,0.00,0.0,64.0,40.0,...,0.00,0.0,73.0,44.0,58.5,67.333333,43.666667,55.500000,0.006667,0.0
3,2023-10-22,0.00,0.0,61.0,42.0,51.5,0.00,0.0,71.0,41.0,...,0.00,0.0,63.0,36.0,49.5,65.000000,39.666667,52.333333,0.000000,0.0
4,2023-10-23,0.00,0.0,67.0,45.0,56.0,0.00,0.0,61.0,38.0,...,0.00,0.0,76.0,43.0,59.5,68.000000,42.000000,55.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,2024-10-16,0.00,0.0,58.0,37.0,47.5,0.03,0.0,54.0,34.0,...,0.00,0.0,60.0,27.0,43.5,57.333333,32.666667,45.000000,0.010000,0.0
364,2024-10-17,0.00,0.0,64.0,38.0,51.0,0.00,0.0,59.0,32.0,...,0.00,NaN,67.0,31.0,49.0,63.333333,33.666667,48.500000,0.000000,0.0
365,2024-10-18,0.00,0.0,69.0,42.0,55.5,0.00,0.0,67.0,34.0,...,0.00,0.0,70.0,38.0,54.0,68.666667,38.000000,53.333333,0.000000,0.0
366,2024-10-19,0.00,0.0,71.0,44.0,57.5,0.00,0.0,71.0,38.0,...,0.00,0.0,74.0,36.0,55.0,72.000000,39.333333,55.666667,0.000000,0.0


In [ ]:
weather_df.columns

Index(['Date', 'Precipitation (in) Chicago', 'Snowfall (in) Chicago',
       'Max Temp (F) Chicago', 'Min Temp (F) Chicago', 'Mean Temp (F) Chicago',
       'Precipitation (in) Champaign', 'Snowfall (in) Champaign',
       'Max Temp (F) Champaign', 'Min Temp (F) Champaign',
       'Mean Temp (F) Champaign', 'Precipitation (in) Springfield',
       'Snowfall (in) Springfield', 'Max Temp (F) Springfield',
       'Min Temp (F) Springfield', 'Mean Temp (F) Springfield',
       'Average Max Temp (F)', 'Average Min Temp (F)', 'Average Mean Temp (F)',
       'Average Precipitation (in)', 'Average Snowfall (in)'],
      dtype='object')

In [ ]:
create_pickle = False

# Merge dataframes on Date
df = dfs["actual_energy_price"]
for key in ["forecasted_load", "cleared_load", "actual_load"]:
    df = pd.merge(df, dfs[key], on='Date')

# Convert Date to pandas timestamp for time delta manipulation
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
df['merge'] = df['Date'].dt.strftime('%m %y')

# Merge with retail price data
df = pd.merge(df, dfs["retail_price"], on='merge')
df.drop(columns=['merge'], inplace=True)

# Calculate kilowatt-hours and pickle the final DataFrame
df['kwhrs'] = df['Actual Price'] / df['Retail Price(cents/kwhr)']

# Merge weather data
df['merge'] = df['Date'].dt.strftime('%m %d %y')
weather_df['merge'] = weather_df['Date'].dt.strftime('%m %d %y')
weather_df.drop(columns = ['Date'], inplace = True)
df = pd.merge(df, weather_df, on='merge')
df.drop(columns=['merge'], inplace=True)

if create_pickle:
  df.to_pickle('/content/drive/MyDrive/DeepDiveProject/data/combined_data.pkl')

  print("Data processing complete and saved to 'drive/MyDrive/DeepDiveProject/data/combined_data.pkl'")

df

,Date,Actual Price,Forecasted Load,Cleared Load,Actual Load,Retail Price(cents/kwhr),kwhrs,Precipitation (in) Chicago,Snowfall (in) Chicago,Max Temp (F) Chicago,...,Precipitation (in) Springfield,Snowfall (in) Springfield,Max Temp (F) Springfield,Min Temp (F) Springfield,Mean Temp (F) Springfield,Average Max Temp (F),Average Min Temp (F),Average Mean Temp (F),Average Precipitation (in),Average Snowfall (in)
0,2023-10-20 16:00:00,41.50,71410,72599,73812,15.75,2.634921,0.0,0.0,61.0,...,0.00,0.0,68.0,45.0,56.5,62.666667,46.333333,54.500000,0.083333,0.0
1,2023-10-20 17:00:00,92.50,71201,73940,73858,15.75,5.873016,0.0,0.0,61.0,...,0.00,0.0,68.0,45.0,56.5,62.666667,46.333333,54.500000,0.083333,0.0
2,2023-10-20 18:00:00,31.93,71200,76048,73281,15.75,2.027302,0.0,0.0,61.0,...,0.00,0.0,68.0,45.0,56.5,62.666667,46.333333,54.500000,0.083333,0.0
3,2023-10-20 19:00:00,26.58,71319,77348,73087,15.75,1.687619,0.0,0.0,61.0,...,0.00,0.0,68.0,45.0,56.5,62.666667,46.333333,54.500000,0.083333,0.0
4,2023-10-20 20:00:00,27.59,70508,73893,71105,15.75,1.751746,0.0,0.0,61.0,...,0.00,0.0,68.0,45.0,56.5,62.666667,46.333333,54.500000,0.083333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6427,2024-07-31 18:00:00,50.56,117585,117182,115967,15.35,3.293811,0.0,0.0,92.0,...,0.32,0.0,82.0,70.0,76.0,87.333333,70.000000,78.666667,0.106667,0.0
6428,2024-07-31 19:00:00,27.55,115484,116193,112798,15.35,1.794788,0.0,0.0,92.0,...,0.32,0.0,82.0,70.0,76.0,87.333333,70.000000,78.666667,0.106667,0.0
6429,2024-07-31 20:00:00,32.36,111855,114754,108578,15.35,2.108143,0.0,0.0,92.0,...,0.32,0.0,82.0,70.0,76.0,87.333333,70.000000,78.666667,0.106667,0.0
6430,2024-07-31 21:00:00,35.01,108298,109081,105087,15.35,2.280782,0.0,0.0,92.0,...,0.32,0.0,82.0,70.0,76.0,87.333333,70.000000,78.666667,0.106667,0.0


In [ ]:
df.columns

Index(['Date', 'Actual Price', 'Forecasted Load', 'Cleared Load',
       'Actual Load', 'Retail Price(cents/kwhr)', 'kwhrs',
       'Precipitation (in) Chicago', 'Snowfall (in) Chicago',
       'Max Temp (F) Chicago', 'Min Temp (F) Chicago', 'Mean Temp (F) Chicago',
       'Precipitation (in) Champaign', 'Snowfall (in) Champaign',
       'Max Temp (F) Champaign', 'Min Temp (F) Champaign',
       'Mean Temp (F) Champaign', 'Precipitation (in) Springfield',
       'Snowfall (in) Springfield', 'Max Temp (F) Springfield',
       'Min Temp (F) Springfield', 'Mean Temp (F) Springfield',
       'Average Max Temp (F)', 'Average Min Temp (F)', 'Average Mean Temp (F)',
       'Average Precipitation (in)', 'Average Snowfall (in)'],
      dtype='object')